# **GCut**

## **Functions**

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")

from morphan import MorphAn

import skimage.io as io
import numpy as np
import multiprocessing as mp

## **GCut Preprocess**
---

#### *Determine Parameters*

* Edit "image_path" variable with a path to an image that you want to evaluate
* Edit "target_slice" variable with a row index that you want to evaluate

In [2]:
MA = MorphAn()

# a path to the experiment
pipeline_dir = os.path.join(module_path,'src','example','analysis_pipeline')
vaa_dir = MA.check_directory(pipeline_dir,'vaa3d')

gcut_dir = MA.check_directory(pipeline_dir,'gcut')
    
trace_dir = MA.check_directory(pipeline_dir,'trace')

epsilon = 35
eps = int(np.floor(epsilon/2))

In [4]:

for image_name in os.listdir(vaa_dir):
    gcut_image_dir = MA.check_directory(gcut_dir,image_name)
    trace_image_dir = MA.check_directory(trace_dir,image_name)
    
    somas = []
    swcs_f = []
    swcs_i = []
    fnames = []
    
    vaa_image_dir = os.path.join(vaa_dir,image_name)
    for var in os.listdir(vaa_image_dir):
        if 'tif' in var or 'txt' in var: continue
            
        fpath = os.path.join(vaa_image_dir,var)
        vaa_swc = MA.read_swc(fpath)
        
        if len(vaa_swc) == 0:
            print("%s has empty tracing."%var)
            continue
        
        res_f = [tuple(map(float,sub.split(' ')[2:4])) for sub in vaa_swc]
        res_i = tuple(tuple(map(int, tup)) for tup in res_f)
        
        x = res_i[0][0]
        y = res_i[0][1]
        
        somas.append((x,y))
        swcs_f.append(res_f)
        swcs_i.append(res_i)
        fnames.append(var)
        
    for si,(swc,fname) in enumerate(zip(swcs_f, fnames)):
        print(ind*2, fname)
        xs = [x[0] for x in swc]
        ys = [x[1] for x in swc]
        src = os.path.join(vaa_dir,fname)

        found_somas = []
        for soma in somas:
            sx,sy = soma

            for ei in range(eps):

                founds = [(x,y) for x,y in zip(xs,ys) if (x <= sx + ei) & (x >= sx - ei) & (y <= sy + ei) & (y >= sy - ei)]
                if founds:
                    fx = founds[0][0]
                    fy = founds[0][1]
                    print(ind*4, sx,sy, '----->', fx,fy)
                    found_somas.append((fx,fy))
                    break

        if len(found_somas) == 1:
            # Single Neuron
            dest = os.path.join(trace_image_dir, fname)
            shutil.copyfile(src,dest)

        elif len(found_somas) > 1:
            # Multiple Neurons
            dest = os.path.join(gcut_image_dir, fname)
            shutil.copyfile(src,dest)
            tmp = fname.split('.swc')[0] + '_soma_ind_fixed.txt'
            gcut_soma = os.path.join(gcut_image_dir, tmp)
            f = open(gcut_soma, 'w', newline='')
            for found_soma in found_somas:
                f.write('%s %s 1\n'%(found_soma[0], found_soma[1]))
            f.close()

## **GCut Process**
---

In [5]:
MA = MorphAn()

# a path to the experiment
pipeline_dir = os.path.join(module_path,'src','example','analysis_pipeline')

gcut_dir = MA.check_directory(pipeline_dir,'gcut')

trace_dir = MA.check_directory(pipeline_dir,'trace')

gcut_path = os.path.join(module_path,'gcut','python','neuron_segmentation.py')

In [6]:

for image_name in os.listdir(gcut_dir):
    
    trace_image_dir = MA.check_directory(trace_dir,image_name)
    gcut_image_dir = MA.check_directory(gcut_dir,image_name)
    for var in os.listdir(gcut_image_dir):
        if 'txt' in var: continue
            
        print(var)

        fname = var.split('.swc')[0]
        swc_path = os.path.join(gcut_image_dir,var)
        soma_path = os.path.join(gcut_image_dir,'%s_soma_ind_fixed.txt'%fname)

        MA.gcut_process(gcut_path,swc_path,soma_path)
    
        src_dir = MA.check_directory(gcut_image_dir,fname)
        if not src_directory:
            print(' '*2, 'No GCut output.')
            continue
            
        for t in os.listdir(src_dir):
            if 'soma=1' not in t: continue
            
            print(' '*2, 'Processed... %s'%t)
            src = os.path.join(src_dir,t)
            dest = os.path.join(trace_image_dir,t)
            shutil.copyfile(src,dest)
            
        print()